In [ ]:
from timeseries_fastai.imports import *
from timeseries_fastai.data import *
from timeseries_fastai.core import *
from timeseries_fastai.models import *

In [ ]:
PATH = get_ucr()

In [ ]:
from ucr import main

In [ ]:
PATH.ls()

(#136) [Path('/home/tc256760/.fastai/data/Univariate_arff/PowerCons'),Path('/home/tc256760/.fastai/data/Univariate_arff/PhalangesOutlinesCorrect'),Path('/home/tc256760/.fastai/data/Univariate_arff/Herring'),Path('/home/tc256760/.fastai/data/Univariate_arff/ElectricDevices'),Path('/home/tc256760/.fastai/data/Univariate_arff/CBF'),Path('/home/tc256760/.fastai/data/Univariate_arff/Plane'),Path('/home/tc256760/.fastai/data/Univariate_arff/DataSummaryExpanded_v03.xlsx'),Path('/home/tc256760/.fastai/data/Univariate_arff/MiddlePhalanxTW'),Path('/home/tc256760/.fastai/data/Univariate_arff/.~lock.DataSummaryExpanded_NoMissingVals.xlsx#'),Path('/home/tc256760/.fastai/data/Univariate_arff/ECG5000')...]

In [ ]:
summary = pd.read_csv(PATH/'SummaryData.csv', index_col=0)

In [ ]:
tasks = summary.index.to_list()

In [ ]:
def max_bs(N):
    N = N//6
    k=1
    while (N//2**k)>1: k+=1
    return min(2**k, 32)

In [ ]:
def get_dls(path, task, bs=None, workers=None):
    df_train, df_test = load_df_ucr(path, task)
    bs = ifnone(bs, max_bs(len(df_train)))
    x_cols = df_train.columns[0:-1].to_list()
    return TSDataLoaders.from_dfs(df_train, df_test, x_cols=x_cols, label_col='target', 
                                  bs=bs, num_workers=workers)

In [ ]:
dls = get_dls(PATH, 'Adiac')

Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Adiac


In [ ]:
def get_model(dls, arch):
    num_classes = len(dls.vocab)
    arch = arch.lower()
    if arch=='resnet':     model = create_resnet(1, num_classes, conv_sizes=[64, 128, 256])
    elif arch=='fcn':      model = create_fcn(1, num_classes, ks=9, conv_sizes=[128, 256, 128])
    elif arch=='mlp':      model = create_mlp(dls.train_ds[0][0].shape[1], num_classes)
    elif arch=='inception':model = create_inception(1, num_classes)
    else: 
        print('Please chosse a model in [resnet, FCN, MLP, inception]')
        return None
    return model

In [ ]:
def train_task(path, task='Adiac', arch='resnet', epochs=40, lr=5e-4):
    "trains arch over task with params"
    dls = get_dls(path, task)
    model = get_model(dls, arch)
    learn = Learner(dls, model, wd=1e-2, metrics=[accuracy])
    learn.fit_one_cycle(epochs, lr)                          
    return learn

In [ ]:
learn = train_task(PATH, 'Adiac', arch='mlp', epochs=1)

Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,3.439373,3.561857,0.074169,00:01


In [ ]:
non_nan_tasks = """Adiac
ArrowHead
Beef
BeetleFly
BirdChicken
Car
CBF
ChlorineConcentration
CinCECGTorso
Coffee
Computers
CricketX
CricketY
CricketZ
DiatomSizeReduction
DistalPhalanxOutlineAgeGroup
DistalPhalanxOutlineCorrect
DistalPhalanxTW
Earthquakes
ECG200
ECG5000
ECGFiveDays
ElectricDevices
FaceAll
FaceFour
FacesUCR
FiftyWords
Fish
FordA
FordB
GunPoint
Ham
HandOutlines
Haptics
Herring
InlineSkate
InsectWingbeatSound
ItalyPowerDemand
LargeKitchenAppliances
Lightning2
Lightning7
Mallat
Meat
MedicalImages
MiddlePhalanxOutlineAgeGroup
MiddlePhalanxOutlineCorrect
MiddlePhalanxTW
MoteStrain
NonInvasiveFatalECGThorax1
NonInvasiveFatalECGThorax2
OliveOil
OSULeaf
PhalangesOutlinesCorrect
Phoneme
Plane
ProximalPhalanxOutlineAgeGroup
ProximalPhalanxOutlineCorrect
ProximalPhalanxTW
RefrigerationDevices
ScreenType
ShapeletSim
ShapesAll
SmallKitchenAppliances
SonyAIBORobotSurface1
SonyAIBORobotSurface2
StarLightCurves
Strawberry
SwedishLeaf
Symbols
SyntheticControl
ToeSegmentation1
ToeSegmentation2
Trace
TwoLeadECG
TwoPatterns
UWaveGestureLibraryAll
UWaveGestureLibraryX
UWaveGestureLibraryY
UWaveGestureLibraryZ
Wafer
Wine
WordSynonyms
Worms
WormsTwoClass
Yoga
ACSF1
BME
EthanolLevel
FreezerRegularTrain
FreezerSmallTrain
GunPointAgeSpan
GunPointMaleVersusFemale
GunPointOldVersusYoung
InsectEPGRegularTrain
InsectEPGSmallTrain
PigAirwayPressure
PigArtPressure
PigCVP
PowerCons
SmoothSubspace
UMD
Fungi
HouseTwenty
SemgHandGenderCh2
SemgHandMovementCh2
SemgHandSubjectCh2
MixedShapes
MixedShapesSmallTrain
EOGHorizontalSignal
EOGVerticalSignal
Rock
Crop
Chinatown
MelbournePedestrian""".split()

In [ ]:
def compute_metrics(learn):
    "compute oguiza Metrics on UCR"
    results = np.array(learn.recorder.values)
    acc_ = results[-1,-1]
    accmax_ = results[:, -1].max()
    loss_ = results[-1,0]
    val_loss_ = results[-1,1]
    return acc_, accmax_, loss_, val_loss_ 

In [ ]:
learn = train_task(PATH, tasks[0], epochs=4)

Loading files from: /home/tc256760/.fastai/data/Univariate_arff/ACSF1


epoch,train_loss,valid_loss,accuracy,time
0,2.022209,2.272052,0.130000,00:01
1,1.616853,2.018378,0.390000,00:01
2,1.377368,1.681458,0.460000,00:01
3,1.221826,1.339833,0.640000,00:01


In [ ]:
compute_metrics(learn)

(0.6399999856948853,
 0.6399999856948853,
 1.2218255996704102,
 1.3398334980010986)

In [ ]:
def run_tasks(tasks, arch='resnet', lr=1e-3, epochs=1):
    results = [compute_metrics(train_task(PATH, task, arch, epochs, lr)) for task in tasks]
    return pd.DataFrame(data=results, columns=['acc', 'acc_max', 'train_loss', 'val_loss'], index=tasks)

In [ ]:
run_tasks(non_nan_tasks[0:3])

Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Adiac


epoch,train_loss,valid_loss,accuracy,time
0,2.797986,4.502527,0.025575,00:01


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/ArrowHead


epoch,train_loss,valid_loss,accuracy,time
0,1.013532,1.219332,0.274286,00:02


Loading files from: /home/tc256760/.fastai/data/Univariate_arff/Beef


epoch,train_loss,valid_loss,accuracy,time
0,1.822442,1.827858,0.200000,00:01


,acc,acc_max,train_loss,val_loss
Adiac,0.025575,0.025575,2.797986,4.502527
ArrowHead,0.274286,0.274286,1.013532,1.219332
Beef,0.200000,0.200000,1.822442,1.827858
